In [1]:
#%%
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from battery_paper import BatteryPaper
from battery_pro import BatteryPro
from battery_regression import BatteryRegression
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import seaborn

#%%
'''
Defining constants..
'''
path_dataset = '../preprocessed/preprocessed_user1.csv'

types = {'milliseconds': np.float64,# np.dtype(int),
         'index': np.int64,# np.dtype(int),
     'day': np.int64,# np.dtype(int),
     'hour': np.int64,# np.dtype(int),
     'battery_level' : np.int64,# np.dtype(int),
     'battery_plugged': np.int64,# np.dtype(int),
     'bright-level': np.int64,# np.dtype(int),
     'screen_on' : np.int64,# np.dtype(int),
     'conn': np.int64,# np.dtype(int),
     'conn_wifi': np.int64,# np.dtype(int),
     'temperature' : np.int64,# np.dtype(int),
     'voltage': np.int64,# np.dtype(int),
     'battery_level_grouped': np.int64,# np.dtype(int),
     'battery_common_smooth': np.int64,# np.dtype(int),
     'battery_exp_smooth': np.int64,# np.dtype(int),
     'no_day': np.int64, 
     'relative_day': np.int64, 
     'relative_min': np.int64,
     }

columns = [u'milliseconds', u'day', u'hour', u'battery_level', u'battery_plugged',
       u'bright_level', u'screen_on', u'conn', u'conn_wifi', u'temperature',
       u'voltage', u'battery_level_grouped', u'battery_common_smooth',
       u'battery_exp_smooth', u'no_day', u'relative_day', u'relative_min']

#%%
'''
Functions to be used..
'''
def load_dataset(path_dataset):
    data_set = pd.read_csv(path_dataset, names=columns)#, dtype=types)
    
    return data_set


def find_pos_by_day(data_set, day, hour):
    days = np.where((data_set['day']==day) & (data_set['hour'] == hour))[0]
                                           
    length = min([len(days), 10])
    return days[:length]
    

def mean_square_error(y_true, y_hat):
    error = np.subtract(y_true, y_hat)
    square_error = np.multiply(error, error)
    
    return round(np.sqrt(sum(np.divide(square_error, len(error)))), 2)
    
def generating_stats(data_set, models, day, hour, user, initial_pos, steps=100):
    fin = initial_pos + steps
    mins = np.array(data_set['relative_min'][initial_pos:fin])
    y_true = np.array(data_set['battery_exp_smooth'][initial_pos:fin])
    
    paper_pred = models[0].predict_level(mins, initial_pos=initial_pos)
    
    pro_pred = models[1].predict_level(mins, initial_pos=initial_pos)
    
    #regr_pred_simple, _ = battery_regression.predict_level(initial_pos=initial_pos,\
    #                                           steps=steps, other_models=False)
    
    regr_pred, _ = models[2].predict_level(initial_pos=initial_pos, steps=steps,
                                                    other_models=True)
    
    row = dict()
    row['user'] = user
    row['day'] = day
    row['hour'] = hour
    row['paper'] = mean_square_error(y_true, paper_pred)
    row['pro'] = mean_square_error(y_true, pro_pred)
    #row['r_simple'] = mean_square_error(y_true, regr_pred_simple)
    row['regr'] = mean_square_error(y_true, regr_pred)
    
    #row[('R2', 'paper')] = r2_score(y_true, paper_pred)
    #row[('R2', 'r_other')] = r2_score(y_true, regr_pred)
    row['pos'] = initial_pos
    
    #name = 'user:{0}_day:{1}_hour:{2}_steps:{3}.png'.format(user, day, hour, steps)
    #plot_curves(paper_pred, pro_pred, regr_pred_simple, regr_pred, y_true,
    #            mins, name)
    return row

def plot_stats(stats, users, days, pdf):
    means = stats.groupby(['user', 'day']).mean()[['paper', 'pro', 'r_simple', 'r_other']]
    pos = range(4)
    labels = ['paper', 'pro', 'r_simple', 'r_other']
    rows = len(users)
    cols = len(days)
    
    f, axarr = plt.subplots(nrows=rows, ncols=cols, figsize=(12, 8), sharey='row')
    plt.setp(axarr, xticks=np.add(pos, 0.4), xticklabels=labels)
    
    cols_headers = ['Day {}'.format(col) for col in days]
    rows_headers = ['{}'.format(header) for header in users]
    
    for ax, col in zip(axarr[0], cols_headers):
        ax.set_title(col)
    
    for ax, row in zip(axarr[:,0], rows_headers):
        ax.set_ylabel(row, rotation=90, size='large')
        
    for row in range(rows):
        for col in range(cols):
            axarr[row, col].bar(pos, means.loc[(users[row], days[col])])
    
    f.tight_layout()
    plt.savefig(pdf, format='pdf')


def plot_curves(paper_pred, pro_pred, regr_pred_simple, regr_pred, 
                y_true, mins, name):
    methods = ['paper', 'pro', 'r_simple', 'r_other']
        
    f, axarr = plt.subplots(nrows=4, ncols=1, figsize=(12, 8))
    
    rows_headers = ['{}'.format(header) for header in methods]
        
    for ax, row in zip(axarr, rows_headers):
        ax.set_ylabel(row, rotation=90, size='large')
    
    axarr[0].plot(mins, paper_pred, 'r',
                     mins, y_true, 'b')
    axarr[1].plot(mins, pro_pred, 'r',
                     mins, y_true, 'b')
    axarr[2].plot(mins, regr_pred_simple, 'r',
                     mins, y_true, 'b')
    axarr[3].plot(mins, regr_pred, 'r',
                     mins, y_true, 'b')
    
    f.tight_layout()
    f.savefig(name)
    

#%%
users = ['user1']#, 'user2', 'user3', 'user4', 'user5', 'user6']
days = [1, 2, 3, 4, 5, 6]
hours = [60 * 12]#[x for x in range(0,1439,60)]
l_steps = [300]
#pp = PdfPages('stats.pdf')

for steps in l_steps:
    stats = pd.DataFrame(columns=['user', 'day', 'paper', 'pro',
                                  'regr', 'pos'])
    for user in users:
        print('Processing {0}'.format(user))
        path_dataset = "../preprocessed/preprocessed_{0}.csv".format(user)#'../preprocessed/preprocessed_{0}.csv'.format(user)
        data_set = load_dataset(path_dataset)
        
        diff = np.subtract(data_set['battery_exp_smooth'][1:], data_set['battery_exp_smooth'][:-1])
        diff = np.array(np.append(diff, [0]))
        plugged = [1 if x > 0 or y == 100 else 0 for (x, y) in zip(diff, data_set['battery_exp_smooth'])]
        data_set['battery_plugged'] = plugged
        
        data_set = data_set.drop_duplicates([u'day', u'hour', u'battery_level', u'battery_exp_smooth'])
        
        # We train the models
        # Paper model
        battery_paper = BatteryPaper()
        battery_paper.fit(data_set)
        # Battery-Pro model
        battery_pro = BatteryPro()
        battery_pro.fit_by_time(data_set, mins=10)
        # Battery regression model
        battery_regression = BatteryRegression()
        battery_regression.fit(data_set)
        
        models = [battery_paper, battery_pro, battery_regression]
        
        for day in days:
            print('--Day {0}'.format(day))
            for hour in hours:
                initial_pos_array = find_pos_by_day(data_set, day, hour)
                for initial_pos in initial_pos_array:
                    print('----Initial pos {0}'.format(initial_pos))
                    # We generate the stats and cumulate them in a DF
                    if initial_pos + steps < data_set.shape[0]:
                        row = generating_stats(data_set, models, day, hour, user, initial_pos, steps)
                        stats = stats.append(row, ignore_index=True)
                
    #plot_stats(stats, users, days, pp)
    
#pp.close()
stats.to_csv('stats.csv')
grouped = stats.groupby(['user', 'day', 'hour'])
print(stats)

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Processing user1


/usr/local/lib/python2.7/dist-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


--Day 1
----Initial pos 6818


battery_regression.py:197: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  prediction = max(min(float(self.__regr__.predict(state_to_predict.reshape(1, -1))[0]), 100), 0)


----Initial pos 16601
----Initial pos 25785
----Initial pos 34927
----Initial pos 44131
----Initial pos 52264
----Initial pos 61283
----Initial pos 69187
----Initial pos 77414
----Initial pos 85986
--Day 2
----Initial pos 8258
----Initial pos 18041
----Initial pos 27016
----Initial pos 45396
----Initial pos 53695
----Initial pos 62349
----Initial pos 70469
----Initial pos 78645
----Initial pos 91344
----Initial pos 99606
--Day 3
----Initial pos 1058
----Initial pos 9698
----Initial pos 19481
----Initial pos 28237
----Initial pos 37479
----Initial pos 46498
----Initial pos 54970
----Initial pos 63523
----Initial pos 71391
----Initial pos 79748
--Day 4
----Initial pos 2498
----Initial pos 11134
----Initial pos 20586
----Initial pos 29448
----Initial pos 38665
----Initial pos 47687
----Initial pos 56311
----Initial pos 64646
----Initial pos 72495
----Initial pos 80846
--Day 5
----Initial pos 3938
----Initial pos 12574
----Initial pos 22017
----Initial pos 30888
----Initial pos 40081
----I

In [2]:
stats.groupby('day').mean()

,paper,pro,regr,pos,hour
day,,,,,
1.0,33.261,21.396,62.779,47439.6,720.0
2.0,35.446,16.969,63.066,55481.9,720.0
3.0,31.997,24.441,57.478,41208.3,720.0
4.0,38.367,24.047,65.737,42431.6,720.0
5.0,37.526,18.949,66.249,43780.6,720.0
6.0,35.249,17.559,60.957,53358.7,720.0
